# Graph Encoding

In [16]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from constants import HOLE_CARD_DICT
import seaborn as sns

In [14]:
hands = hand_chart.values.ravel()

In [26]:
hands

array(['AAo', 'AKs', 'AQs', 'AJs', 'ATs', 'A9s', 'A8s', 'A7s', 'A6s',
       'A5s', 'A4s', 'A3s', 'A2s', 'AKo', 'KKo', 'KQs', 'KJs', 'KTs',
       'K9s', 'K8s', 'K7s', 'K6s', 'K5s', 'K4s', 'K3s', 'K2s', 'AQo',
       'KQo', 'QQo', 'QJs', 'QTs', 'Q9s', 'Q8s', 'Q7s', 'Q6s', 'Q5s',
       'Q4s', 'Q3s', 'Q2s', 'AJo', 'KJo', 'QJo', 'JJo', 'JTs', 'J9s',
       'J8s', 'J7s', 'J6s', 'J5s', 'J4s', 'J3s', 'J2s', 'ATo', 'KTo',
       'QTo', 'JTo', 'TTo', 'T9s', 'T8s', 'T7s', 'T6s', 'T5s', 'T4s',
       'T3s', 'T2s', 'A9o', 'K9o', 'Q9o', 'J9o', 'T9o', '99o', '98s',
       '97s', '96s', '95s', '94s', '93s', '92s', 'A8o', 'K8o', 'Q8o',
       'J8o', 'T8o', '98o', '88o', '87s', '86s', '85s', '84s', '83s',
       '82s', 'A7o', 'K7o', 'Q7o', 'J7o', 'T7o', '97o', '87o', '77o',
       '76s', '75s', '74s', '73s', '72s', 'A6o', 'K6o', 'Q6o', 'J6o',
       'T6o', '96o', '86o', '76o', '66o', '65s', '64s', '63s', '62s',
       'A5o', 'K5o', 'Q5o', 'J5o', 'T5o', '95o', '85o', '75o', '65o',
       '55o', '54s',

In [2]:
df = pd.read_csv("data/hole_card_data.csv")

In [38]:
df.head()

,hand,river_win_rate,connectedness,suited,pair,hand_idx,quantile_bins
0,22o,0.491883,0,0,1,0,1
1,32o,0.293683,1,0,0,1,0
2,32s,0.330875,1,1,0,2,0
3,33o,0.528633,0,0,1,3,2
4,42o,0.302725,2,0,0,4,0


In [42]:
starting_hands = set(df.hand.tolist())

In [43]:
starting_hands

{'22o',
 '32o',
 '32s',
 '33o',
 '42o',
 '42s',
 '43o',
 '43s',
 '44o',
 '52o',
 '52s',
 '53o',
 '53s',
 '54o',
 '54s',
 '55o',
 '62o',
 '62s',
 '63o',
 '63s',
 '64o',
 '64s',
 '65o',
 '65s',
 '66o',
 '72o',
 '72s',
 '73o',
 '73s',
 '74o',
 '74s',
 '75o',
 '75s',
 '76o',
 '76s',
 '77o',
 '82o',
 '82s',
 '83o',
 '83s',
 '84o',
 '84s',
 '85o',
 '85s',
 '86o',
 '86s',
 '87o',
 '87s',
 '88o',
 '92o',
 '92s',
 '93o',
 '93s',
 '94o',
 '94s',
 '95o',
 '95s',
 '96o',
 '96s',
 '97o',
 '97s',
 '98o',
 '98s',
 '99o',
 'A2o',
 'A2s',
 'A3o',
 'A3s',
 'A4o',
 'A4s',
 'A5o',
 'A5s',
 'A6o',
 'A6s',
 'A7o',
 'A7s',
 'A8o',
 'A8s',
 'A9o',
 'A9s',
 'AAo',
 'AJo',
 'AJs',
 'AKo',
 'AKs',
 'AQo',
 'AQs',
 'ATo',
 'ATs',
 'J2o',
 'J2s',
 'J3o',
 'J3s',
 'J4o',
 'J4s',
 'J5o',
 'J5s',
 'J6o',
 'J6s',
 'J7o',
 'J7s',
 'J8o',
 'J8s',
 'J9o',
 'J9s',
 'JJo',
 'JTo',
 'JTs',
 'K2o',
 'K2s',
 'K3o',
 'K3s',
 'K4o',
 'K4s',
 'K5o',
 'K5s',
 'K6o',
 'K6s',
 'K7o',
 'K7s',
 'K8o',
 'K8s',
 'K9o',
 'K9s',
 'KJo',


In [ ]:
hand_groups = {
    "Premium": {},
    "Strong": {},
    "Suited Connectors": {},
    "Low Pocket Pairs": {},
    "Suited One-Gappers": {},
    "Offsuit Connectors": {},
    "Garbage": {},
    "Speculative": {}
}

In [12]:
pd.cut(df['river_win_rate'], bins=10).value_counts()

river_win_rate
(0.517, 0.572]    34
(0.405, 0.461]    30
(0.349, 0.405]    29
(0.461, 0.517]    29
(0.572, 0.628]    20
(0.293, 0.349]    13
(0.628, 0.684]     7
(0.795, 0.851]     3
(0.684, 0.739]     2
(0.739, 0.795]     2
Name: count, dtype: int64

In [30]:
df['quantile_bins'] = pd.cut(df['river_win_rate'], bins=5, labels=range(5))

In [37]:
df[df['quantile_bins'] == 2]

,hand,river_win_rate,connectedness,suited,pair,hand_idx,quantile_bins
3,33o,0.528633,0,0,1,3,2
8,44o,0.560300,0,0,1,8,2
15,55o,0.596183,0,0,1,15,2
24,66o,0.625850,0,0,1,24,2
64,A2o,0.529300,1,0,0,64,2
65,A2s,0.556375,1,1,0,65,2
66,A3o,0.538208,2,0,0,66,2
67,A3s,0.563700,2,1,0,67,2
68,A4o,0.547400,3,0,0,68,2
69,A4s,0.567050,3,1,0,69,2
